In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
df = pd.read_csv("paragrapghs_with_topics_banks.csv", index_col=0)
# df = pd.read_csv("../topics/paragrapghs_with_topics.csv.zip", index_col=0)
df.head()

,Unnamed: 0,index,company,year,paragraph,original_text,cleaned_text,Экология в целом,Климат,Энергия,...,Корпоративное управление_base_words_count,Устойчивое развитие_percent_of_words,Устойчивое развитие_importance,Устойчивое развитие_base_words_count,!Кибербезопасность_percent_of_words,!Кибербезопасность_importance,!Кибербезопасность_base_words_count,cleaned_texttopics,original_texttopics,count_words
0,0,1481,ПАО Сбербанк,2020,0,годовой отчет 2020,годовой отчёт 2020,0.0,0.0,0.0,...,0.0,0.333333,0.0,1.0,0.0,0.0,0.0,годовой отчёт 2020,годовой отчет 2020,NaN
1,1,1481,ПАО Сбербанк,2020,1,содержание,содержание,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,содержание,содержание,NaN
2,2,1481,ПАО Сбербанк,2020,2,содержание 3,содержание 3,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,содержание 3,содержание 3,NaN
3,3,1481,ПАО Сбербанк,2020,3,обращение председателя правления оао«бпс-сберб...,обращение председатель правление,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,обращение председатель правление,обращение председателя правления оао«бпс-сберб...,NaN
4,4,1481,ПАО Сбербанк,2020,4,27,27,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
not_nan = df[df["max_topic_exp"].notna()]
# число абзацев соотвествующих теме
reports = not_nan.groupby(["year", "company", "index", "max_topic_exp"])["paragraph"].count()
reports.head()

year  company            index  max_topic_exp           
2020  АКБ "Держава" ПАО  2738   Вода                         3
                                Закупки и антикоррупция     11
                                Корпоративное управление     6
                                Обучение и развитие         18
                                Оплата труда                27
Name: paragraph, dtype: int64

# TF-IDF

## TF

In [4]:
topics_per_report = not_nan.groupby(["year", "index"])["paragraph"].count()
topics_per_report.head()

year  index
2020  1         811
      121       477
      170        79
      354      1551
      415       345
Name: paragraph, dtype: int64

In [5]:
tf = defaultdict(dict)
for (year, file_name, index, topic), row in reports.items():
    tf[(year, index)][topic] = row / topics_per_report[year][index]
tf = pd.DataFrame(tf).T

## IDF

In [6]:
tf

Вода  Закупки и антикоррупция  Корпоративное управление  \
2020 2738  0.011811                 0.043307                  0.023622   
     1326  0.038835                      NaN                  0.029126   
     170        NaN                 0.012658                  0.037975   
     2816  0.056911                 0.016260                       NaN   
     2748       NaN                 0.016949                       NaN   
...             ...                      ...                       ...   
2022 485        NaN                 0.006536                       NaN   
     493   0.022222                      NaN                       NaN   
     2225  0.005128                 0.015385                       NaN   
     2272  0.002904                 0.028074                  0.000968   
     1481  0.010893                 0.018154                  0.000908   

           Обучение и развитие  Оплата труда  Отношения с инвесторами  \
2020 2738             0.070866      0.106299                 0.212598   
     1326             0.174757           NaN                 0.126214   
     170              0.101266      0.101266                 0.202532   
     2816             0.081301      0.056911                 0.308943   
     2748             0.033898      0.028249                 0.107345   
...                        ...           ...                      ...   
2022 485              0.104575      0.013072                 0.385621   
     493              0.022222      0.155556                 0.111111   
     2225             0.205128      0.071795                 0.051282   
     2272             0.222652      0.018393                 0.030010   
     1481             0.157035      0.027231                 0.050227   

           Отходы и циклическая экономика  Поставщики в целом  \
2020 2738                        0.023622            0.043307   
     1326                             NaN                 NaN   
     170                         0.025316            0.012658   
     2816                             NaN            0.040650   
     2748                             NaN            0.011299   
...                                   ...                 ...   
2022 485                              NaN            0.039216   
     493                         0.044444                 NaN   
     2225                             NaN            0.041026   
     2272                             NaN            0.034850   
     1481                        0.002118            0.067171   

           Потребители. Качество и безопас  Потребители. Сервис и коммуника  \
2020 2738                         0.098425                         0.011811   
     1326                         0.048544                         0.135922   
     170                          0.012658                              NaN   
     2816                         0.065041                         0.065041   
     2748                         0.129944                         0.158192   
...                                    ...                              ...   
2022 485                          0.071895                         0.084967   
     493                          0.044444                         0.044444   
     2225                         0.020513                         0.082051   
     2272                         0.029042                         0.067764   
     1481                         0.054463                         0.103782   

           ...  Потребители. Доступность  Потребители. Персональные данны  \
2020 2738  ...                       NaN                              NaN   
     1326  ...                       NaN                              NaN   
     170   ...                       NaN                              NaN   
     2816  ...                       NaN                              NaN   
     2748  ...                       NaN                              NaN   
...        ...                       ...                 

In [7]:
idf = {}
for topic in df["max_topic_exp"].unique():
    if pd.isna(topic):
        continue
    idf[topic] = np.log(df["index"].nunique() / tf[topic].notna().sum())
idf = pd.Series(idf)

In [8]:
tf_idf = defaultdict(dict)
for idx, row in tf.iterrows():
    for topic in row.index:
        tf_idf[idx][topic] = row[topic] * idf[topic]

In [9]:
tf_idf = pd.DataFrame(tf_idf).T
tf_idf = tf_idf.fillna(0)
tf_idf.head()

Вода  Закупки и антикоррупция  Корпоративное управление  \
2020 2738 -0.004688                -0.016435                  0.007827   
     1326 -0.015413                 0.000000                  0.009651   
     170   0.000000                -0.004804                  0.012583   
     2816 -0.022587                -0.006171                  0.000000   
     2748  0.000000                -0.006432                  0.000000   

           Обучение и развитие  Оплата труда  Отношения с инвесторами  \
2020 2738            -0.035102     -0.045792                -0.108601   
     1326            -0.086561      0.000000                -0.064473   
     170             -0.050159     -0.043624                -0.103458   
     2816            -0.040270     -0.024516                -0.157816   
     2748            -0.016791     -0.012169                -0.054834   

           Отходы и циклическая экономика  Поставщики в целом  \
2020 2738                        0.013524           -0.019372   
     1326                        0.000000            0.000000   
     170                         0.014494           -0.005662   
     2816                        0.000000           -0.018183   
     2748                        0.000000           -0.005054   

           Потребители. Качество и безопас  Потребители. Сервис и коммуника  \
2020 2738                        -0.048752                        -0.005283   
     1326                        -0.024045                        -0.060800   
     170                         -0.006270                         0.000000   
     2816                        -0.032216                        -0.029093   
     2748                        -0.064364                        -0.070761   

           ...  Потребители. Доступность  Потребители. Персональные данны  \
2020 2738  ...                       0.0                              0.0   
     1326  ...                       0.0                              0.0   
     170   ...                       0.0                              0.0   
     2816  ...                       0.0                              0.0   
     2748  ...                       0.0                              0.0   

           Сотрудники. Наем и увольнение  Потребители. Здоровье и благопо  \
2020 2738                            0.0                              0.0   
     1326                            0.0                              0.0   
     170                             0.0                              0.0   
     2816                            0.0                              0.0   
     2748                            0.0                              0.0   

           Устойчивое развитие  Воздух  Отчетность и прозрачность  \
2020 2738                  0.0     0.0                        0.0   
     1326                  0.0     0.0                        0.0   
     170                   0.0     0.0                        0.0   
     2816                  0.0     0.0                        0.0   
     2748                  0.0     0.0                        0.0   

           Сотрудники. Волонтерство  Этика  Экологичность продукта  
2020 2738                       0.0    0.0                     0.0  
     1326                       0.0    0.0                     0.0  
     170                        0.0    0.0                     0.0  
     2816                       0.0    0.0                     0.0  
     2748                       0.0    0.0                     0.0  

[5 rows x 37 columns]

In [10]:
tf_idf.to_csv("tf_idf.csv")

In [11]:
topic_words = pd.read_csv("../topic_words.csv", index_col=0)
topic_words.head()

,topic,weight,type,word
meta,,,,
env,Экология в целом,5.0,0.0,экологичный
env,Экология в целом,5.0,0.0,природоохранный
env,Экология в целом,1.0,0.0,экология
env,Экология в целом,5.0,0.0,воздействие
env,Экология в целом,5.0,0.0,окружающая среда


In [12]:
gr = topic_words.groupby("meta")["topic"].apply(set)

# Second

In [13]:
from pymcdm.methods import TOPSIS

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [14]:
gr = (
    df.drop(
        columns=[
            "Unnamed: 0",
            "paragraph",
            "original_text",
            "cleaned_text",
            "max_topic_cos",
            "max_score_cos",
            "count_words",
            "second_max_topic_cos",
            "second_max_score_cos",
            "max_topic_exp",
            "max_score_exp",
            "second_max_topic_exp",
            "second_max_score_exp",
        ]
    )
    .groupby(["year", "company", "index"])
    .mean()
    .dropna(how="all")
)
gr.head()

/tmp/ipykernel_85206/2231286138.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .mean()


Экология в целом    Климат   Энергия  \
year company               index                                         
2020 АКБ "Держава" ПАО     2738                0.0  0.000000  0.002566   
     АО "АЛЬФА-БАНК"       1326                0.0  0.002193  0.001113   
     АО "Авто Финанс Банк" 170                 0.0  0.000000  0.000000   
     АО "БАНК СГБ"         2816                0.0  0.000000  0.006611   
     АО "БМ-Банк"          2748                0.0  0.000000  0.000000   

                                    Воздух      Вода  \
year company               index                       
2020 АКБ "Держава" ПАО     2738   0.000521  0.001269   
     АО "АЛЬФА-БАНК"       1326   0.000000  0.006358   
     АО "Авто Финанс Банк" 170    0.000000  0.003083   
     АО "БАНК СГБ"         2816   0.000000  0.004302   
     АО "БМ-Банк"          2748   0.000420  0.009929   

                                  Отходы и циклическая экономика  \
year company               index                                   
2020 АКБ "Держава" ПАО     2738                         0.004549   
     АО "АЛЬФА-БАНК"       1326                         0.005941   
     АО "Авто Финанс Банк" 170                          0.001776   
     АО "БАНК СГБ"         2816                         0.002507   
     АО "БМ-Банк"          2748                         0.001525   

                                  Биоразнообразие  Рекультивация земель  \
year company               index                                          
2020 АКБ "Держава" ПАО     2738          0.000802              0.000664   
     АО "АЛЬФА-БАНК"       1326          0.001862              0.001702   
     АО "Авто Финанс Банк" 170           0.001923              0.000000   
     АО "БАНК СГБ"         2816          0.002026              0.000000   
     АО "БМ-Банк"          2748          0.001302              0.001032   

                                  Экологичность продукта  !Персонал в целом  \
year company               index                                              
2020 АКБ "Держава" ПАО     2738                 0.005595                0.0   
     АО "АЛЬФА-БАНК"       1326                 0.003946                0.0   
     АО "Авто Финанс Банк" 170                  0.000000                0.0   
     АО "БАНК СГБ"         2816                 0.002898                0.0   
     АО "БМ-Банк"          2748                 0.002753                0.0   

                                  ...  Этика_base_words_count  \
year company               index  ...                           
2020 АКБ "Держава" ПАО     2738   ...                0.124454   
     АО "АЛЬФА-БАНК"       1326   ...                0.036290   
     АО "Авто Финанс Банк" 170    ...                0.078534   
     АО "БАНК СГБ"         2816   ...                0.040441   
     АО "БМ-Банк"          2748   ...                0.040564   

                                  Корпоративное управление_percent_of_words  \
year company               index                                              
2020 АКБ "Держава" ПАО     2738                                    0.059431   
     АО "АЛЬФА-БАНК"       1326                                    0.015274   
     АО "Авто Финанс Банк" 170                                     0.045097   
     АО "БАНК СГБ"         2816                                    0.014315   
     АО "БМ-Банк"          2748                                    0.005491   

                                  Корпоративное управление_importance  \
year company               index                                        
2020 АКБ "Держава" ПАО     2738                              1.211790   
     АО "АЛЬФА-БАНК"       1326                              0.294355   
     АО "Авто Финанс Банк" 170                               0.502618   
     АО "БАНК СГБ"         2816                              0.169118   
     АО "БМ-Банк"          2748                              0.028219   

                                  Корпорати

In [15]:
topics = topic_words.groupby("meta")["topic"].apply(set)

In [16]:
gr.reset_index(inplace=True)
gr = gr.set_index("index")
gr.head()

,year,company,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,...,Этика_base_words_count,Корпоративное управление_percent_of_words,Корпоративное управление_importance,Корпоративное управление_base_words_count,Устойчивое развитие_percent_of_words,Устойчивое развитие_importance,Устойчивое развитие_base_words_count,!Кибербезопасность_percent_of_words,!Кибербезопасность_importance,!Кибербезопасность_base_words_count
index,,,,,,,,,,,,,,,,,,,,,
2738,2020,"АКБ ""Держава"" ПАО",0.0,0.000000,0.002566,0.000521,0.001269,0.004549,0.000802,0.000664,...,0.124454,0.059431,1.211790,2.037118,0.019971,0.187773,1.683406,0.000949,0.0,0.052402
1326,2020,"АО ""АЛЬФА-БАНК""",0.0,0.002193,0.001113,0.000000,0.006358,0.005941,0.001862,0.001702,...,0.036290,0.015274,0.294355,0.491935,0.027354,0.217742,0.705645,0.000275,0.0,0.008065
170,2020,"АО ""Авто Финанс Банк""",0.0,0.000000,0.000000,0.000000,0.003083,0.001776,0.001923,0.000000,...,0.078534,0.045097,0.502618,0.769634,0.029525,0.146597,2.136126,0.000658,0.0,0.041885
2816,2020,"АО ""БАНК СГБ""",0.0,0.000000,0.006611,0.000000,0.004302,0.002507,0.002026,0.000000,...,0.040441,0.014315,0.169118,0.356618,0.058691,0.073529,0.779412,0.000903,0.0,0.022059
2748,2020,"АО ""БМ-Банк""",0.0,0.000000,0.000000,0.000420,0.009929,0.001525,0.001302,0.001032,...,0.040564,0.005491,0.028219,0.195767,0.023385,0.033510,0.945326,0.000061,0.0,0.001764


In [17]:
def second_workflow(data: pd.DataFrame):
    matrix = data.to_numpy()
    weights = np.array([1 / matrix.shape[0]] * matrix.shape[1])
    types = np.ones(matrix.shape[1], dtype=int)
    body = TOPSIS()
    predictions = [round(preference, 2) for preference in body(matrix, weights, types)]
    return predictions

In [18]:
bad = ["Кибербезопасность", "Инновации", "Персонал в целом"]

for year in gr["year"].unique():
    year_data = gr[gr["year"] == year]
    for key, cur_topics in topics.items():
        tmp_topics = []
        for t in cur_topics:
            if t not in bad:
                tmp_topics.append(t)
            else:
                tmp_topics.append("!" + t)
        cur_topics = tmp_topics
        res = second_workflow(year_data[cur_topics])
        res = pd.Series(res).set_axis(year_data.index)
        gr.loc[gr["year"] == year, key] = res

In [19]:
gr["avg"] = (gr["env"] + gr["gov"] + gr["social"]) / 3
gr["on_letters"] = second_workflow(gr[["env", "gov", "social"]])
gr = gr.sort_values(["year", "avg"], ascending=[True, False])
gr.head()

,year,company,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,...,Устойчивое развитие_importance,Устойчивое развитие_base_words_count,!Кибербезопасность_percent_of_words,!Кибербезопасность_importance,!Кибербезопасность_base_words_count,env,gov,social,avg,on_letters
index,,,,,,,,,,,,,,,,,,,,,
1978,2020,"ПАО ""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК""",0.006946,0.004917,0.005623,0.004854,0.008042,0.004554,0.001790,0.002236,...,0.341107,1.064015,0.005184,0.0,0.064472,0.44,0.35,0.55,0.446667,0.67
2590,2020,"ПАО ""АК БАРС"" БАНК",0.028138,0.002301,0.002888,0.016649,0.015065,0.002624,0.000879,0.000564,...,0.177197,0.676010,0.004073,0.0,0.055107,0.50,0.28,0.55,0.443333,0.65
2272,2020,ПАО РОСБАНК,0.000000,0.000000,0.009549,0.001790,0.000000,0.002408,0.003993,0.006850,...,0.543478,2.934783,0.002504,0.0,0.094203,0.47,0.39,0.39,0.416667,0.63
1000,2020,Банк ВТБ (ПАО),0.017305,0.003102,0.003155,0.011062,0.012205,0.002650,0.002739,0.001134,...,0.164085,0.555881,0.002323,0.0,0.028045,0.45,0.24,0.48,0.390000,0.57
963,2020,"ПАО ""Совкомбанк""",0.023276,0.006965,0.002361,0.015026,0.014051,0.000616,0.000581,0.000926,...,0.221734,0.457286,0.002235,0.0,0.033291,0.50,0.24,0.37,0.370000,0.53


In [23]:
bad = ["Кибербезопасность", "Инновации", "Персонал в целом"]

for year in gr["year"].unique():
    year_data = gr[gr["year"] == year]
    tmp_topics = []
    for t in [v for _, val in topics.items() for v in val]:
        if t not in bad:
            tmp_topics.append(t)
        else:
            tmp_topics.append("!" + t)
    cur_topics = tmp_topics
    res = second_workflow(year_data[cur_topics])
    res = pd.Series(res).set_axis(year_data.index)
    gr.loc[gr["year"] == year, "all_letters"] = res

In [24]:
for year in gr["year"].unique():
    gr[gr["year"] == year].to_csv(f"banks_results_{year}.csv")
gr.to_csv("banks_results.csv")

In [22]:
res.shape

(16,)